https://chat.openai.com/share/1f5ffa88-c3a2-46d3-8e68-2ee41187d02e



# Installing Dependancies

In [ ]:
# Install and import RDKit
%%capture
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdChemReactions
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
import numpy as np


# Parsing Compounds and Populating **EcoCyc Chemical Class**

Upload compounds.dat using GDown

In [ ]:
import gdown

file_id = "1mB-xIewC1UY3mLzVi0ovRV6bQPS7RzYg"
output_file = "compounds.dat"

gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file)

#Alternative method if gdown doesn't work
#from google.colab import files
#uploaded = files.upload()

Downloading...
From: https://drive.google.com/uc?id=1mB-xIewC1UY3mLzVi0ovRV6bQPS7RzYg
To: /content/compounds.dat
100%|██████████| 8.34M/8.34M [00:00<00:00, 29.2MB/s]


'compounds.dat'

Using a parser for the chemical ontology for the chemcial class and dictionary. This is retrieved from **compounds.dat**



In [ ]:
class EcoCyc_Chemical:
    def __init__(self, unique_id="", common_name="", smiles="", inchi=""):
        self.UNIQUE_ID = unique_id
        self.COMMON_NAME = common_name
        self.SMILES = smiles
        self.INCHI = inchi

    def __repr__(self):
        return f"EcoCyc_Chemical(UNIQUE_ID={self.UNIQUE_ID}, COMMON_NAME={self.COMMON_NAME}, SMILES={self.SMILES}, INCHI={self.INCHI})"

def parse_chemicals_to_dict(file_contents):
    ecocyc_chemicals_dict = {}
    ecocyc_chemical_data = {}

    for line in file_contents:
        if '- ' in line:
            key, value = line.split('- ', 1) #1 is used as a max split to limit the number of splits
            key = key.strip()
            value = value.strip()

            if key == 'UNIQUE-ID':
                # Example of parser: If the key is 'UNIQUE-ID', store the value in chemical_data with the key 'unique_id'
                ecocyc_chemical_data['unique_id'] = value
            elif key == 'SMILES':
                ecocyc_chemical_data['smiles'] = value
            elif key == 'INCHI':
                ecocyc_chemical_data['inchi'] = value
            elif key == 'COMMON-NAME':
                ecocyc_chemical_data['common_name'] = value
        elif line.strip() == '//':  # End of chemical data entry
            chem_obj = EcoCyc_Chemical(**ecocyc_chemical_data) #creates Chemical object useing the collected data stored in 'chemical_data' dictionary
            ecocyc_chemicals_dict[ecocyc_chemical_data['unique_id']] = chem_obj #Adds newly created Chemical object to chemicals_dict with the unique_id used as the key
            ecocyc_chemical_data = {}  # Reset for the next entry

    return ecocyc_chemicals_dict

# Assuming you've uploaded the file 'compounds.dat' to Colab
with open("compounds.dat", "r", encoding="ISO-8859-1") as file:
    file_contents = file.readlines()

ecocyc_chemicals_dict = parse_chemicals_to_dict(file_contents)

# Test by printing the first 5 chemicals
for key in list(ecocyc_chemicals_dict.keys())[:5]:
    print(ecocyc_chemicals_dict[key])


EcoCyc_Chemical(UNIQUE_ID=Ureidocarboxylates, COMMON_NAME=an ureidocarboxylate, SMILES=, INCHI=)
EcoCyc_Chemical(UNIQUE_ID=Monogalactosyldiacylglycerols-38-3, COMMON_NAME=monogalactosyldiacylglycerol-38:3, SMILES=, INCHI=)
EcoCyc_Chemical(UNIQUE_ID=6-H-5-I-2-methylhexanoates, COMMON_NAME=a 6-hydroxy-5-isopropenyl-2-methylhexanoate, SMILES=C=C(C)C(CCC(C)C(=O)[O-])CO, INCHI=InChI=1S/C10H18O3/c1-7(2)9(6-11)5-4-8(3)10(12)13/h8-9,11H,1,4-6H2,2-3H3,(H,12,13)/p-1)
EcoCyc_Chemical(UNIQUE_ID=Monogalactosyldiacylglycerols-36-4, COMMON_NAME=monogalactosyldiacylglycerol-36:4, SMILES=, INCHI=)
EcoCyc_Chemical(UNIQUE_ID=Short-Chain-Carboxylates, COMMON_NAME=a short-chain carboxylate, SMILES=C([O-])(=O)[R], INCHI=)


# Parsing Reactions and Populating **EcoCyc Reaction Class**

Upload reactions.dat using GDown

In [ ]:
import gdown

file_id = "1SjVODv98B3KJxV0hFrNAPN0B-3nTfr1b"
output_file = "reactions.dat"

gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file)

#Alternative method if gdown doesn't work
#from google.colab import files
#uploaded = files.upload()

Downloading...
From: https://drive.google.com/uc?id=1SjVODv98B3KJxV0hFrNAPN0B-3nTfr1b
To: /content/reactions.dat
100%|██████████| 1.66M/1.66M [00:00<00:00, 134MB/s]


'reactions.dat'

Using a parser for the reaction ontology for the reaction class and dictionary. This is retrieved from **reactions.dat**



In [ ]:
class EcoCyc_Reaction:
    def __init__(self, unique_id="", left=None, right=None, reaction_direction=""):
        self.UNIQUE_ID = unique_id
        self.LEFT = left if left is not None else []
        self.RIGHT = right if right is not None else []
        self.REACTION_DIRECTION = reaction_direction

    def __repr__(self):
        return f"EcoCyc_Reaction(UNIQUE_ID={self.UNIQUE_ID}, LEFT={self.LEFT}, RIGHT={self.RIGHT}, REACTION_DIRECTION={self.REACTION_DIRECTION})"

def parse_reactions_to_dict(file_contents):
    ecocyc_reactions_dict = {}
    ecocyc_reaction_data = {
        'left': [],
        'right': []
    }

    for line in file_contents:
        if '- ' in line:
            key, value = line.split('- ', 1)
            key = key.strip()
            value = value.strip()

            if key == 'UNIQUE-ID':
                ecocyc_reaction_data['unique_id'] = value
            elif key == 'LEFT':
                ecocyc_reaction_data['left'].append(value)
            elif key == 'RIGHT':
                ecocyc_reaction_data['right'].append(value)
            elif key == 'REACTION-DIRECTION':
                ecocyc_reaction_data['reaction_direction'] = value
        elif line.strip() == '//':  # End of reaction data entry
            reaction_obj = EcoCyc_Reaction(**ecocyc_reaction_data)
            ecocyc_reactions_dict[ecocyc_reaction_data['unique_id']] = reaction_obj
            # Reset for the next entry
            ecocyc_reaction_data = {
                'left': [],
                'right': []
            }

    return ecocyc_reactions_dict

with open("reactions.dat", "r", encoding="ISO-8859-1") as file:
    file_contents = file.readlines()

ecocyc_reactions_dict = parse_reactions_to_dict(file_contents)

# Test by printing the first 5 reactions
for key in list(ecocyc_reactions_dict.keys())[:5]:
    print(ecocyc_reactions_dict[key])



EcoCyc_Reaction(UNIQUE_ID=RXN-21279, LEFT=['a-5-deoxyribose-5-phosphate-DNA'], RIGHT=['5-Phospho-terminated-DNAs', 'CPD-22978', 'PROTON'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT)
EcoCyc_Reaction(UNIQUE_ID=RXN-19954, LEFT=['2Cys-Peroxiredoxins-With-HydroxyCys'], RIGHT=['Cys2-Peroxiredoxin-Disulfide', 'WATER'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT)
EcoCyc_Reaction(UNIQUE_ID=RXN-17919, LEFT=['A-5-prime-PP-5-prime-DNA', '3-Hydroxy-Terminated-DNAs'], RIGHT=['DNA-N', 'AMP', 'PROTON'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT)
EcoCyc_Reaction(UNIQUE_ID=RXN0-4701, LEFT=['mRNA-Holder', 'WATER'], RIGHT=['ssRNA-with-3phosphate', 'ssRNA-with-5OH'], REACTION_DIRECTION=LEFT-TO-RIGHT)
EcoCyc_Reaction(UNIQUE_ID=RXN-20692, LEFT=['Cys2-Peroxiredoxin-Disulfide', 'Red-Prx-Disulfide-Reductases'], RIGHT=['Reduced-Cys2-Peroxiredoxins', 'Ox-Prx-Disulfide-Reductases'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT)


#Structural Specificity Filtering to Create **Chemical Class**

This text will describe the new class and dictionary.

In [ ]:
class Chemical:
    def __init__(self, unique_id="", common_name="", smiles="", inchi="", structure_specificity=None):
        self.UNIQUE_ID = unique_id
        self.COMMON_NAME = common_name
        self.SMILES = smiles
        self.INCHI = inchi
        self.STRUCTURE_SPECIFICITY = structure_specificity

    def __repr__(self):
        return f"Chemical(UNIQUE_ID={self.UNIQUE_ID}, COMMON_NAME={self.COMMON_NAME}, SMILES={self.SMILES}, INCHI={self.INCHI}, STRUCTURE_SPECIFICITY={self.STRUCTURE_SPECIFICITY})"

# Conversion of the old dictionary to the new format
chemicals_dict = {}
for unique_id, ecocyc_chemical in ecocyc_chemicals_dict.items():
    chemical = Chemical(unique_id=ecocyc_chemical.UNIQUE_ID,
                                         common_name=ecocyc_chemical.COMMON_NAME,
                                         smiles=ecocyc_chemical.SMILES,
                                         inchi=ecocyc_chemical.INCHI)
    chemicals_dict[unique_id] = chemical

# Now, you can use the classify_chemicals function to update the STRUCTURE_SPECIFICITY field
# (The classify_chemicals function and process_for_chirality function should be defined as provided)

Creating the chemical filter using functionality from RDkit

In [ ]:
# Function to classify chemicals based on the given criteria
def classify_chemicals(chemicals_dict):
    for chem_id, chemical in chemicals_dict.items():
        # Check if SMILES is absent
        if not chemical.SMILES:
            chemical.STRUCTURE_SPECIFICITY = "class"
            continue

        # Try to create a Mol object from SMILES
        mol = Chem.MolFromSmiles(chemical.SMILES)

        # If Mol object is created from SMILES
        if mol:
            # Process for chiral centers and define structure specificity
            process_for_chirality(chemical, mol)
            continue

        # If Mol object not created from SMILES, try from SMARTS
        mol = Chem.MolFromSmarts(chemical.SMILES)

        # If Mol object is created from SMARTS
        if mol:
            chemical.STRUCTURE_SPECIFICITY = "RSMILES"
        else:
            chemical.STRUCTURE_SPECIFICITY = "abstract"

def process_for_chirality(chemical, mol):
    mol = Chem.AddHs(mol)  # Add hydrogens
    chiral_centers = Chem.FindMolChiralCenters(mol, force=True, includeUnassigned=True)

    if chiral_centers:
        if any(chirality == '?' for center, chirality in chiral_centers):
            chemical.STRUCTURE_SPECIFICITY = "stereoundefined"
        else:
            chemical.STRUCTURE_SPECIFICITY = "concrete"
    else:
        chemical.STRUCTURE_SPECIFICITY = "concrete"  # No chiral centers implies concrete?



Running the filter

In [ ]:
# Call the function to classify the chemicals
classify_chemicals(chemicals_dict)

# Test by printing the classified chemicals
for key, chemical in chemicals_dict.items():
    print(chemical)


#OPTIONAL SUMMARIZING STEPS BELOW

# Step 1: Count the occurrences of each structure specificity category
category_counts = {"class": 0, "concrete": 0, "stereoundefined": 0, "RSMILES": 0, "abstract": 0}

for chem in chemicals_dict.values():
    if chem.STRUCTURE_SPECIFICITY:
        category_counts[chem.STRUCTURE_SPECIFICITY] += 1

# Step 2: Display the results
print("\nFilter Success - Structure Specificity Distribution:")
for category, count in category_counts.items():
    print(f"{category}: {count}")

# Step 3: Calculate and display the percentage distribution
total = sum(category_counts.values())
if total > 0:
    print("\nPercentage Distribution:")
    for category, count in category_counts.items():
        percentage = (count / total) * 100
        print(f"{category}: {percentage:.2f}%")

Streaming output truncated to the last 5000 lines.
[21:30:31] SMILES Parse Error: Failed parsing SMILES 'C([NH2+]C1(\C=C/[C@H](O)[C@H](O)1))C2(\C4(C(=O)NC(/N)=N/C(N(/C=2)[C@@H]3(O[C@H](COP(=O)([O-])O[a' for input: 'C([NH2+]C1(\C=C/[C@H](O)[C@H](O)1))C2(\C4(C(=O)NC(/N)=N/C(N(/C=2)[C@@H]3(O[C@H](COP(=O)([O-])O[a'
[21:30:31] SMARTS Parse Error: syntax error while parsing: C([NH2+]C1(\C=C/[C@H](O)[C@H](O)1))C2(\C4(C(=O)NC(/N)=N/C(N(/C=2)[C@@H]3(O[C@H](COP(=O)([O-])O[a
[21:30:31] SMARTS Parse Error: Failed parsing SMARTS 'C([NH2+]C1(\C=C/[C@H](O)[C@H](O)1))C2(\C4(C(=O)NC(/N)=N/C(N(/C=2)[C@@H]3(O[C@H](COP(=O)([O-])O[a' for input: 'C([NH2+]C1(\C=C/[C@H](O)[C@H](O)1))C2(\C4(C(=O)NC(/N)=N/C(N(/C=2)[C@@H]3(O[C@H](COP(=O)([O-])O[a'
[21:30:31] SMILES Parse Error: syntax error while parsing: C([a
[21:30:31] SMILES Parse Error: Failed parsing SMILES 'C([a' for input: 'C([a'
[21:30:31] SMARTS Parse Error: syntax error while parsing: C([a
[21:30:31] SMARTS Parse Error: Failed parsing SMARTS 'C([a' for

Streaming output truncated to the last 5000 lines.
Chemical(UNIQUE_ID=L-glucose, COMMON_NAME=L-glucose, SMILES=C(=O)[C@H]([C@@H]([C@H]([C@H](CO)O)O)O)O, INCHI=InChI=1S/C6H12O6/c7-1-3(9)5(11)6(12)4(10)2-8/h1,3-6,8-12H,2H2/t3-,4+,5+,6+/m1/s1, STRUCTURE_SPECIFICITY=concrete)
Chemical(UNIQUE_ID=Cyclic-Ribonucleoside-Monophosphates, COMMON_NAME=a nucleoside cyclic phosphate, SMILES=, INCHI=, STRUCTURE_SPECIFICITY=class)
Chemical(UNIQUE_ID=L-idose, COMMON_NAME=L-idose, SMILES=C(=O)[C@H](O)[C@@H](O)[C@H](O)[C@@H](O)CO, INCHI=InChI=1S/C6H12O6/c7-1-3(9)5(11)6(12)4(10)2-8/h1,3-6,8-12H,2H2/t3-,4-,5+,6+/m0/s1, STRUCTURE_SPECIFICITY=concrete)
Chemical(UNIQUE_ID=2-Prime-Ribonucleoside-Monophosphates, COMMON_NAME=a ribonucleoside 2'-phosphate, SMILES=C(O)[C@H]1(O[C@@H]([a nucleobase])[C@H](OP([O-])(=O)[O-])[C@H](O)1), INCHI=, STRUCTURE_SPECIFICITY=abstract)
Chemical(UNIQUE_ID=Quinovose, COMMON_NAME=quinovose, SMILES=, INCHI=, STRUCTURE_SPECIFICITY=class)
Chemical(UNIQUE_ID=Nucleotide-Conjugates, COMM

# Creating Reaction Class

This section will convert to a new class with new labels. It will also add a boolean marker for reversible, and it will switch the order of the right to left reactions

In [ ]:
class Reaction(EcoCyc_Reaction):
    def __init__(self, unique_id="", substrates=None, products=None, reaction_direction="", reversible=False):
        super().__init__(unique_id, left=substrates, right=products, reaction_direction=reaction_direction)
        self.SUBSTRATES = substrates if substrates is not None else []
        self.PRODUCTS = products if products is not None else []
        self.REVERSIBLE = reversible

    def __repr__(self):
        return f"Reaction(UNIQUE_ID={self.UNIQUE_ID}, SUBSTRATES={self.SUBSTRATES}, PRODUCTS={self.PRODUCTS}, REACTION_DIRECTION={self.REACTION_DIRECTION}, REVERSIBLE={self.REVERSIBLE})"

# Conversion of the old reactions dictionary (ecocyc_reactions_dict) to the new format (reactions_dict)
reactions_dict = {}
for unique_id, ecocyc_reaction in ecocyc_reactions_dict.items():
    reversible = ecocyc_reaction.REACTION_DIRECTION.upper() == "REVERSIBLE"
    reaction = Reaction(unique_id=ecocyc_reaction.UNIQUE_ID,
                        substrates=ecocyc_reaction.LEFT,
                        products=ecocyc_reaction.RIGHT,
                        reaction_direction=ecocyc_reaction.REACTION_DIRECTION,
                        reversible=reversible)
    reactions_dict[unique_id] = reaction

# Now, reactions_dict contains Reaction objects with the additional 'REVERSIBLE' field


Updating reversible boolean

In [ ]:
# Iterate through all reactions and update the 'reversible' field
for reaction in reactions_dict.values():
    if reaction.REACTION_DIRECTION == "Reversible":
        reaction.REVERSIBLE = True

# Print the first 5 reactions as a sample output
for key in list(reactions_dict.keys())[180:200]:
    print(reactions_dict[key])

Reaction(UNIQUE_ID=TRANS-RXN0-240, SUBSTRATES=['BIOTIN'], PRODUCTS=['BIOTIN'], REACTION_DIRECTION=LEFT-TO-RIGHT, REVERSIBLE=False)
Reaction(UNIQUE_ID=2.7.7.61-RXN, SUBSTRATES=['2-5-TRIPHOSPHORIBOSYL-3-DEPHOSPHO-', 'APO-CITRATE-LYASE'], PRODUCTS=['HOLO-CITRATE-LYASE', 'PPI'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT, REVERSIBLE=False)
Reaction(UNIQUE_ID=TRANS-RXN-146, SUBSTRATES=['CPD0-2279'], PRODUCTS=['CPD0-2279'], REACTION_DIRECTION=LEFT-TO-RIGHT, REVERSIBLE=False)
Reaction(UNIQUE_ID=RXN0-5513, SUBSTRATES=['ATP', 'ACP', 'CPD66-39'], PRODUCTS=['AMP', 'PPI', 'Saturated-Fatty-Acyl-ACPs'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT, REVERSIBLE=False)
Reaction(UNIQUE_ID=TRANS-RXN0-579, SUBSTRATES=['HYPOXANTHINE'], PRODUCTS=['HYPOXANTHINE'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT, REVERSIBLE=False)
Reaction(UNIQUE_ID=1.7.2.2-RXN, SUBSTRATES=['PROTON', 'Cytochromes-C-Reduced', 'NITRITE'], PRODUCTS=['WATER', 'AMMONIUM', 'Cytochromes-C-Oxidized'], REACTION_DIRECTION=LEFT-TO-RIGHT, REVERSIBLE=

Swapping products and reactants with right-to-left labels

In [ ]:
# Initialize a counter
swap_count = 0

# Iterate through all reactions in reactions_dict
for reaction in reactions_dict.values():
    if reaction.REACTION_DIRECTION in ["RIGHT-TO-LEFT", "PHYSIOL-RIGHT-TO-LEFT"]:
        # Swap SUBSTRATES and PRODUCTS
        reaction.SUBSTRATES, reaction.PRODUCTS = reaction.PRODUCTS, reaction.SUBSTRATES
        # Increment the counter
        swap_count += 1

# Print the count of reactions where the swap occurred
print("Number of reactions with direction RIGHT-TO-LEFT or PHYSIOL-RIGHT-TO-LEFT:", swap_count)

# Print a few sample reactions to verify changes
for key in list(reactions_dict.keys())[:500]:
    print(reactions_dict[key])


Number of reactions with direction RIGHT-TO-LEFT or PHYSIOL-RIGHT-TO-LEFT: 179
Reaction(UNIQUE_ID=RXN-21279, SUBSTRATES=['a-5-deoxyribose-5-phosphate-DNA'], PRODUCTS=['5-Phospho-terminated-DNAs', 'CPD-22978', 'PROTON'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT, REVERSIBLE=False)
Reaction(UNIQUE_ID=RXN-19954, SUBSTRATES=['2Cys-Peroxiredoxins-With-HydroxyCys'], PRODUCTS=['Cys2-Peroxiredoxin-Disulfide', 'WATER'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT, REVERSIBLE=False)
Reaction(UNIQUE_ID=RXN-17919, SUBSTRATES=['A-5-prime-PP-5-prime-DNA', '3-Hydroxy-Terminated-DNAs'], PRODUCTS=['DNA-N', 'AMP', 'PROTON'], REACTION_DIRECTION=PHYSIOL-LEFT-TO-RIGHT, REVERSIBLE=False)
Reaction(UNIQUE_ID=RXN0-4701, SUBSTRATES=['mRNA-Holder', 'WATER'], PRODUCTS=['ssRNA-with-3phosphate', 'ssRNA-with-5OH'], REACTION_DIRECTION=LEFT-TO-RIGHT, REVERSIBLE=False)
Reaction(UNIQUE_ID=RXN-20692, SUBSTRATES=['Cys2-Peroxiredoxin-Disulfide', 'Red-Prx-Disulfide-Reductases'], PRODUCTS=['Reduced-Cys2-Peroxiredoxins', 'Ox-Prx-

# Old Parsing Reactions and Populating **Reaction Class**


Upload reactions.dat using GDown

In [ ]:
import gdown

file_id = "1SjVODv98B3KJxV0hFrNAPN0B-3nTfr1b"
output_file = "reactions.dat"

gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file)

Downloading...
From: https://drive.google.com/uc?id=1SjVODv98B3KJxV0hFrNAPN0B-3nTfr1b
To: /content/reactions.dat
100%|██████████| 1.66M/1.66M [00:00<00:00, 42.2MB/s]


'reactions.dat'



```
# This is formatted as code
```

Using a parser for the reaction ontology for the reaction class and dictionary. This is retrieved from **reactions.dat**

In [ ]:
class Reactions:
    def __init__(self, unique_id="", substrates=None, products=None):
        self.UNIQUE_ID = unique_id
        self.SUBSTRATES = substrates if substrates is not None else []
        self.PRODUCTS = products if products is not None else []

    def __repr__(self):
        return f"Reactions(UNIQUE_ID={self.UNIQUE_ID}, SUBSTRATES={self.SUBSTRATES}, PRODUCTS={self.PRODUCTS})"

def parse_reactions_to_dict(file_contents):
    reactions_dict = {}
    reaction_data = {
        'substrates': [],
        'products': []
    }

    for line in file_contents:
        if '- ' in line:
            key, value = line.split('- ', 1)
            key = key.strip()
            value = value.strip()

            if key == 'UNIQUE-ID':
                reaction_data['unique_id'] = value
            elif key == 'LEFT':
                reaction_data['substrates'].append(value)
            elif key == 'RIGHT':
                reaction_data['products'].append(value)
        elif line.strip() == '//':  # End of reaction data entry
            reaction_obj = Reactions(**reaction_data)
            reactions_dict[reaction_data['unique_id']] = reaction_obj
            # Reset for the next entry
            reaction_data = {
                'substrates': [],
                'products': []
            }

    return reactions_dict

with open("reactions.dat", "r", encoding="ISO-8859-1") as file:
    file_contents = file.readlines()

reactions_dict = parse_reactions_to_dict(file_contents)

# Test by printing the first 5 reactions
for key in list(reactions_dict.keys())[:5]:
    print(reactions_dict[key])


Reactions(UNIQUE_ID=RXN-21279, SUBSTRATES=['a-5-deoxyribose-5-phosphate-DNA'], PRODUCTS=['5-Phospho-terminated-DNAs', 'CPD-22978', 'PROTON'])
Reactions(UNIQUE_ID=RXN-19954, SUBSTRATES=['2Cys-Peroxiredoxins-With-HydroxyCys'], PRODUCTS=['Cys2-Peroxiredoxin-Disulfide', 'WATER'])
Reactions(UNIQUE_ID=RXN-17919, SUBSTRATES=['A-5-prime-PP-5-prime-DNA', '3-Hydroxy-Terminated-DNAs'], PRODUCTS=['DNA-N', 'AMP', 'PROTON'])
Reactions(UNIQUE_ID=RXN0-4701, SUBSTRATES=['mRNA-Holder', 'WATER'], PRODUCTS=['ssRNA-with-3phosphate', 'ssRNA-with-5OH'])
Reactions(UNIQUE_ID=RXN-20692, SUBSTRATES=['Cys2-Peroxiredoxin-Disulfide', 'Red-Prx-Disulfide-Reductases'], PRODUCTS=['Reduced-Cys2-Peroxiredoxins', 'Ox-Prx-Disulfide-Reductases'])
